In [18]:
import hashlib
import json
import requests
import pandas as pd



URL = 'https://scanner.tradingview.com/crypto/scan'
body = {
    "filter":[],
    "options":{
        "lang":"en"
    },
    "markets":[
        "crypto"
    ],
    "symbols":{
        "query":{
            "types":[]
        },
        "tickers":[]
    },
    "columns":[
        "name",
        "base_currency",
        "currency",
        "close",
        "exchange",
        "type",
        "subtype"
    ],
    "sort":{
        "sortBy":"24h_vol|5",
        "sortOrder":"desc"
    },
    "price_conversion":{
        "to_symbol":False
    },
    "range":[0,100000]
}


d = pd.DataFrame()
res = requests.post(URL, data=json.dumps(body))

df = dict()
if res.status_code == 200:
    response = json.loads(res.text)
    count = response['totalCount']
    data = [d['d'] if d['d'][1] and d['d'][2] else [] for d in response['data']]
    df = pd.DataFrame(
        data, 
        columns=['NAME', 'CUR1', 'CUR2', 'PRICE1', 'EXCH', 'TYPE', 'SUBTYPE']
    )


In [19]:
exch = [
    'UNISWAP', 'BINANCE', 'MEXC', 'GATEIO',
    'HITBTC', 'UNISWAP3ETH', 'KUCOIN', 'BITTREX',
    'COINEX', 'OKX', 'HUOBI', 'KRAKEN',
    'SUSHISWAP', 'POLONIEX', 'BITMEX', 'BITFINEX',
    'COINBASE', 'BITGET', 'UNISWAP3POLYGON', 'PHEMEX',
    'BITRUE', 'BINANCEUS', 'UPBIT', 'BYBIT',
    'WHITEBIT', 'BITHUMB', 'MERCADO'
]

print(len(df))

# Remove emty rows
df.dropna(subset=['NAME'], inplace=True)

# Remove undesible exchanges
df = df[df.EXCH.isin(exch)]

# Remove where price is 0
df = df[df.PRICE1 != 0]

# Remove rows where CUR1 == CUR2
df = df[df.CUR1 != df.CUR2]

# Remove rows with special pairs
df = df[df.CUR1 + df.CUR2 == df.NAME ]

# Create new col price of reverse currency
df['PRICE2'] = 1 / df['PRICE1']


print(len(df))
# df.to_csv('./test.csv', sep='\t')

39323
20554


In [20]:


# Create reverse CUR1 and CUR2
reverse_df = df.copy()
reverse_df['CUR1'] = df['CUR2']
reverse_df['CUR2'] = df['CUR1']
reverse_df['PRICE1'] = df['PRICE2']
reverse_df['PRICE2'] = df['PRICE1']


# Concatanete df
df = pd.concat([df,reverse_df])
print(len(df))


def hash_unicode(a_string):
    return hashlib.sha256(a_string.encode('utf-8')).hexdigest()

df['HASH'] = df['NAME'] + df['CUR1'] + df['CUR2'] + df['EXCH'] + df['TYPE']
df['HASH'] = df['HASH'].apply(hash_unicode)


41108


In [35]:
import sqlalchemy
from sqlalchemy.sql import text



db_address = 'sqlite:////cross-exchange-arbitrage/db.sqlite3'


cross_exch_query = '''
select tvd.hash_pair from arbitrage_tradingviewdata tvd 
'''
engine = sqlalchemy.create_engine(db_address)
res = False
with engine.connect() as connection:
    res = connection.execute(text(cross_exch_query)).fetchall()

print(res)

OperationalError: (sqlite3.OperationalError) unable to open database file
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [80]:
y_df = df.merge(
    df,
    left_on='CUR1',
    right_on='CUR2'
)

    
# Create single pair arbitrage table
single_df = y_df[y_df.NAME_x == y_df.NAME_y]
single_df = single_df[single_df.EXCH_x != single_df.EXCH_y]

single_df['PROF1'] = round((single_df['PRICE2_y'] * 100 / single_df['PRICE1_x'])-100, 5)
single_df['PROF2'] = round((single_df['PRICE2_y'] * 100 / single_df['PRICE1_x'])-100, 5)


# # # Sort result by profit
single_df = single_df[
    ((single_df.PROF1 >= 1) | 
    (single_df.PROF2 >= 1)) & 
    ((single_df.PROF1 <= 100) |
     (single_df.PROF2 <= 100))
].sort_values('PROF1', ascending=False)
print('Single: ', len(single_df))
single_df

Single:  664


,NAME_x,CUR1_x,CUR2_x,PRICE1_x,EXCH_x,TYPE_x,SUBTYPE_x,PRICE2_x,NAME_y,CUR1_y,CUR2_y,PRICE1_y,EXCH_y,TYPE_y,SUBTYPE_y,PRICE2_y,PROF1,PROF2
6058794,QIUSDT,USDT,QI,58.343057,MEXC,spot,crypto,0.017140,QIUSDT,QI,USDT,0.008790,BINANCE,crypto,defi,113.765643,94.99431,94.99431
12273148,QIUSDT,QI,USDT,0.008790,BINANCE,crypto,defi,113.765643,QIUSDT,USDT,QI,58.343057,MEXC,spot,crypto,0.017140,94.99431,94.99431
12288018,DFLUSDT,DFL,USDT,0.001467,MEXC,spot,crypto,681.663258,DFLUSDT,USDT,DFL,354.735722,GATEIO,spot,crypto,0.002819,92.16087,92.16087
9451034,DFLUSDT,USDT,DFL,354.735722,GATEIO,spot,crypto,0.002819,DFLUSDT,DFL,USDT,0.001467,MEXC,spot,crypto,681.663258,92.16087,92.16087
12273149,QIUSDT,QI,USDT,0.008790,BINANCE,crypto,defi,113.765643,QIUSDT,USDT,QI,60.532688,GATEIO,spot,crypto,0.016520,87.94084,87.94084
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12289912,CRAFTUSDT,CRAFT,USDT,0.020878,GATEIO,spot,crypto,47.897308,CRAFTUSDT,USDT,CRAFT,47.415837,MEXC,spot,crypto,0.021090,1.01542,1.01542
8615830,ARGUSDT,USDT,ARG,0.678426,MEXC,spot,crypto,1.474000,ARGUSDT,ARG,USDT,1.459200,GATEIO,spot,crypto,0.685307,1.01425,1.01425
12289292,ARGUSDT,ARG,USDT,1.459200,GATEIO,spot,crypto,0.685307,ARGUSDT,USDT,ARG,0.678426,MEXC,spot,crypto,1.474000,1.01425,1.01425
12283402,RLYUSDT,RLY,USDT,0.015145,MEXC,spot,crypto,66.028392,RLYUSDT,USDT,RLY,65.372295,GATEIO,spot,crypto,0.015297,1.00363,1.00363


In [81]:
''' Cross-exchange arbitrage multiple pairs'''
import sqlalchemy

engine = sqlalchemy.create_engine('sqlite:///arbitrage.db')
single_df.to_sql('single_arbitrage', engine, if_exists='replace', index=True)

664

In [ ]:

# Create single pair arbitrage table
single_df = y_df[y_df.NAME_x == y_df.NAME_y]
single_df['PROF'] = round((single_df['PRICE_y'] * 100 / single_df['PRICE_x'])-100, 5)

# Sort result by profit
single_df = single_df[single_df.PROF >= 1].sort_values('PROF', ascending=False)
print('Single: ', len(single_df))


print(len(y_df))

In [76]:
data = ('252cdfdb37d1a6985f8beab1b523cee496082edcfbe1aca5372bd117ce903152', 'c4b4f31d2d4520fde2fce5463942671f8121ac8344ddc5ee303e881d7890872d', 'dced8fc20b344c3ebe438ff4b28e17772d63c6727f685ff7cc91db02d64121fb', 7.53824101290202), ('252cdfdb37d1a6985f8beab1b523cee496082edcfbe1aca5372bd117ce903152', 'c8d42d51e29160c99bb989c6b315bd8ad393c53a2073dd5f921f88fb83c03bf0', '6d26a2ecb5025b2cc4c86f9c185a52196b56d427406725e499c529c46a3e4d42', 3.5283804924637394), ('252cdfdb37d1a6985f8beab1b523cee496082edcfbe1aca5372bd117ce903152', 'ca0a9737961a0b13c7a9fb4084e6c66dc2e1a410d4061b2f8b4d22f4718e9687', 'f30c1f8efedfb6ec228804ad3eb225ee5e7c3934b3cfb154538b7114a1d0d336', 2.5882780666118634), ('252cdfdb37d1a6985f8beab1b523cee496082edcfbe1aca5372bd117ce903152', 'cca050b59f6407306324c88eb622b5570c79b8875492e65574ff40fb57e912c3', 'faf1857dc2d06ec4067fa2d5116610c8e81753d5351c8096d4eaa35326ecde00', 6.424003323216624), ('252cdfdb37d1a6985f8beab1b523cee496082edcfbe1aca5372bd117ce903152', 'd0b616b13f0db583820506469ababf282d3b57fdbfca7aa0b0e61ee1bc4fa613', 'd2f991c728b110c0e4431be0127eb51321584a45c278eb1b4a742f5a7199a07f', 27.601539503959316), ('252cdfdb37d1a6985f8beab1b523cee496082edcfbe1aca5372bd117ce903152', 'd2a51dac8cb725420c34eedfe5830129fef917fde0beebb89ce329a541da72ce', '4ba6e951aaf0b272a358c3dd388832a15b6fb0e5119ae912b0bb9365de2d9aef', 6.911437828201144), ('252cdfdb37d1a6985f8beab1b523cee496082edcfbe1aca5372bd117ce903152', 'd6f0880d609a75e3e984976ecdcdb6f3c1ba82687d343cd26369211c2952335e', '90c4b3f13313af1b877e5839ac9ad9f5a562a642c3f0792377cdde71982c24b7', 9.029016534185843), ('252cdfdb37d1a6985f8beab1b523cee496082edcfbe1aca5372bd117ce903152', 'd7e73ea5538b3ed67c90c075409365fb10b747d893678b469ad3d1f8d1360da3', '8b5d8e987e37fc5efc894760cbbc5846a871ab50abea5d293fb4f72ae1bd0fdd', 2.6791235816643137)

test = {
    'c4b4f31d2d4520fde2fce5463942671f8121ac8344ddc5ee303e881d7890872d': 0,
    'c8d42d51e29160c99bb989c6b315bd8ad393c53a2073dd5f921f88fb83c03bf0': 1,
    'ca0a9737961a0b13c7a9fb4084e6c66dc2e1a410d4061b': 2
}
df = pd.DataFrame(data, columns=['1', '2', '3', "4"])

col = df.columns.to_list()
df['HASH'] = df['1'] + df['2']+ df['3']
df['HASH'] = df['HASH'].apply(hash_unicode)

df1 = df.replace({"2":test})
df1

,1,2,3,4,HASH
0,252cdfdb37d1a6985f8beab1b523cee496082edcfbe1ac...,0,dced8fc20b344c3ebe438ff4b28e17772d63c6727f685f...,7.538241,b2f1c5b45333edce974591fc33a8b734eb87d42a16a4b7...
1,252cdfdb37d1a6985f8beab1b523cee496082edcfbe1ac...,1,6d26a2ecb5025b2cc4c86f9c185a52196b56d427406725...,3.528380,5ea6dde0161a744cb90cff58dc68f33b4c56be22bf9da5...
2,252cdfdb37d1a6985f8beab1b523cee496082edcfbe1ac...,ca0a9737961a0b13c7a9fb4084e6c66dc2e1a410d4061b...,f30c1f8efedfb6ec228804ad3eb225ee5e7c3934b3cfb1...,2.588278,12a2e54467e97c42993b66104d0ff6e1bef3dd48379140...
3,252cdfdb37d1a6985f8beab1b523cee496082edcfbe1ac...,cca050b59f6407306324c88eb622b5570c79b8875492e6...,faf1857dc2d06ec4067fa2d5116610c8e81753d5351c80...,6.424003,50093f5ae689f6b9dc34cc80d96b2ff18e654fb99b2773...
4,252cdfdb37d1a6985f8beab1b523cee496082edcfbe1ac...,d0b616b13f0db583820506469ababf282d3b57fdbfca7a...,d2f991c728b110c0e4431be0127eb51321584a45c278eb...,27.601540,36de632c88640cf87c6afb8666345c71f095f7ad998334...
5,252cdfdb37d1a6985f8beab1b523cee496082edcfbe1ac...,d2a51dac8cb725420c34eedfe5830129fef917fde0beeb...,4ba6e951aaf0b272a358c3dd388832a15b6fb0e5119ae9...,6.911438,c2d3023828cdd3df3f66eeca56f6e69923c64540effa32...
6,252cdfdb37d1a6985f8beab1b523cee496082edcfbe1ac...,d6f0880d609a75e3e984976ecdcdb6f3c1ba82687d343c...,90c4b3f13313af1b877e5839ac9ad9f5a562a642c3f079...,9.029017,1f7432093e938ec1f81e1fb1188a6f451985c2a40648fe...
7,252cdfdb37d1a6985f8beab1b523cee496082edcfbe1ac...,d7e73ea5538b3ed67c90c075409365fb10b747d893678b...,8b5d8e987e37fc5efc894760cbbc5846a871ab50abea5d...,2.679124,9d18fc915cad5ffe92cc44d074109fe770df2373849a32...
